In [2]:
import os

from defines import *
from lib.yahoo_transit import *
import folium


In [3]:
MAPBOX_ATTR = """© <a href="https://www.mapbox.com/about/maps/">Mapbox</a> ©
<a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>
<strong
  ><a href="https://labs.mapbox.com/contribute/" target="_blank"
    >Improve this map</a
  ></strong
>"""

In [4]:
BUS_STOPS_OF_DIRECT_ROUTE: list[BusStop] = []
BUS_STOPS_OF_THROUGH_ROUTE: list[BusStop] = []
BUS_STOP_OF_BOTH_ROUTE: list[BusStop] = []

for bus_stop in KAIT_SCHOOL_ZONE_BUS_STOPS:
    # 神工大関係の駅なら直接ぶちこむ
    if INSIDE_KAIT_BUS_STOP == bus_stop:
        BUS_STOPS_OF_DIRECT_ROUTE.append(bus_stop)
        continue
    if FRONT_OF_KAIT_BUS_STOP == bus_stop:
        BUS_STOPS_OF_THROUGH_ROUTE.append(bus_stop)
        continue

    is_able_direct = is_able_to_reach_from_either(INSIDE_KAIT_BUS_STOP, bus_stop, 0)
    is_able_through = is_able_to_reach_from_either(FRONT_OF_KAIT_BUS_STOP, bus_stop, 0)
    # どっちも辿り着ける?
    if is_able_direct and is_able_through:
        BUS_STOP_OF_BOTH_ROUTE.append(bus_stop)
    # 学内バス停に辿り着ける?    
    if is_able_direct:
        BUS_STOPS_OF_DIRECT_ROUTE.append(bus_stop)
    # 学校前バス停に辿り着ける?
    if is_able_through:
        BUS_STOPS_OF_THROUGH_ROUTE.append(bus_stop)


In [13]:
folium_map = folium.Map(
    location=KAIT_GEOMETRY.folium_coordinate(),
    zoom_start=13,
    tiles="https://cyberjapandata.gsi.go.jp/xyz/std/{z}/{x}/{y}.png",
    attr="出典: 国土地理院ウェブサイト・地理院タイル・標準地図 " + MAPBOX_ATTR
)

color = "red"
for bs in BUS_STOPS_OF_DIRECT_ROUTE:
    if bs == INSIDE_KAIT_BUS_STOP:
        folium.Marker(
            location=bs.geometry.folium_coordinate(), 
            icon=folium.Icon(color=color, prefix="fa", icon="school")
        ).add_to(folium_map)
    else:
        if bs not in BUS_STOP_OF_BOTH_ROUTE:
            folium.Marker(
                location=bs.geometry.folium_coordinate(), 
                icon=folium.Icon(color=color, prefix="fa", icon="bus")
            ).add_to(folium_map)
        else:
            folium.Marker(
                location=bs.geometry.folium_coordinate(), 
                icon=folium.Icon(color="green", prefix="fa", icon="bus")
            ).add_to(folium_map)

color = "blue"
for bs in BUS_STOPS_OF_THROUGH_ROUTE:
    if bs == FRONT_OF_KAIT_BUS_STOP:
        folium.Marker(
            location=bs.geometry.folium_coordinate(), 
            icon=folium.Icon(color=color, prefix="fa", icon="school")
        ).add_to(folium_map)
    else:
        if bs not in BUS_STOP_OF_BOTH_ROUTE:
            folium.Marker(
                location=bs.geometry.folium_coordinate(), 
                icon=folium.Icon(color=color, prefix="fa", icon="bus")
            ).add_to(folium_map)
        else:
            folium.Marker(
                location=bs.geometry.folium_coordinate(), 
                icon=folium.Icon(color="green", prefix="fa", icon="bus")
            ).add_to(folium_map)

folium_map

In [8]:
from shapely import unary_union
import geopandas
from dotenv import load_dotenv
import os
from lib.mapbox import *
from lib.geometry import *
from lib.busstop import BusStop 

load_dotenv()
mapbox = MapBoxApi(os.environ["MAPBOX_API_TOKEN"])

folium_map = folium.Map(
    location=KAIT_GEOMETRY.folium_coordinate(),
    zoom_start=13,
    tiles="https://cyberjapandata.gsi.go.jp/xyz/std/{z}/{x}/{y}.png",
    attr="出典: 国土地理院ウェブサイト・地理院タイル・標準地図 " + MAPBOX_ATTR
)

# AREA
area_10min = []
area_20min = []
area_30min = []
dones = []
for stop in BUS_STOP_OF_BOTH_ROUTE+BUS_STOPS_OF_THROUGH_ROUTE+BUS_STOPS_OF_DIRECT_ROUTE:
    if stop in dones:
        continue
    dones.append(stop)
    area = mapbox.get_isochrone(
        prof=IsochroneProfile.Walking,
        geo=stop.geometry,
        contours_minutes=[10, 20, 30],
    )
    gdf = geopandas.GeoDataFrame.from_features(area)
    area_10min.append(gdf.geometry[2])
    area_20min.append(gdf.geometry[1])
    area_30min.append(gdf.geometry[0])
area_10min_polygon = geopandas.GeoSeries(unary_union(area_10min))
area_20min_polygon = geopandas.GeoSeries(unary_union(area_20min))
area_30min_polygon = geopandas.GeoSeries(unary_union(area_30min))

folium.GeoJson(
    area_10min_polygon.to_json(),
).add_to(folium_map)
folium.GeoJson(
    area_20min_polygon.to_json(),
).add_to(folium_map)
folium.GeoJson(
    area_30min_polygon.to_json(),
).add_to(folium_map)



# PIN
color = "red"
for bs in BUS_STOPS_OF_DIRECT_ROUTE:
    if bs == INSIDE_KAIT_BUS_STOP:
        folium.Marker(
            location=bs.geometry.folium_coordinate(), 
            icon=folium.Icon(color=color, prefix="fa", icon="school")
        ).add_to(folium_map)
    else:
        if bs not in BUS_STOP_OF_BOTH_ROUTE:
            folium.Marker(
                location=bs.geometry.folium_coordinate(), 
                icon=folium.Icon(color=color, prefix="fa", icon="bus")
            ).add_to(folium_map)
        else:
            folium.Marker(
                location=bs.geometry.folium_coordinate(), 
                icon=folium.Icon(color="green", prefix="fa", icon="bus")
            ).add_to(folium_map)

color = "blue"
for bs in BUS_STOPS_OF_THROUGH_ROUTE:
    if bs == FRONT_OF_KAIT_BUS_STOP:
        folium.Marker(
            location=bs.geometry.folium_coordinate(), 
            icon=folium.Icon(color=color, prefix="fa", icon="school")
        ).add_to(folium_map)
    else:
        if bs not in BUS_STOP_OF_BOTH_ROUTE:
            folium.Marker(
                location=bs.geometry.folium_coordinate(), 
                icon=folium.Icon(color=color, prefix="fa", icon="bus")
            ).add_to(folium_map)
        else:
            folium.Marker(
                location=bs.geometry.folium_coordinate(), 
                icon=folium.Icon(color="green", prefix="fa", icon="bus")
            ).add_to(folium_map)
folium_map.save("yt_reachable_area.html")
folium_map